# Subspace Similarity

## Setup

In [ ]:
# --- Core Libraries for Data Handling and Numerics ---
import io
import re
import os
import json
import zipfile
import requests
import shutil
import subprocess
from typing import List, Tuple, Dict, Optional

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# --- TensorFlow and TF-Hub for Word2Vec ---
import tensorflow as tf
import tensorflow_hub as hub

# --- PyTorch and Hugging Face Transformers for BERT ---
import torch
from transformers import AutoTokenizer, AutoModel

print("All libraries imported successfully.")

All libraries imported successfully.


In [ ]:
### 1) TF-Hub Wiki-words (word2vec-style) wrapper
W2V_HANDLE = "https://tfhub.dev/google/Wiki-words-500-with-normalization/2"

def init_w2v_embedder(handle: str = W2V_HANDLE):
    """
    Returns a simple callable embed(words: List[str]) -> np.ndarray (float32).
    Uses hub.load() which returns a callable SavedModel for this TF2 handle.
    """
    try:
        module = hub.load(handle)   # TF2 saved model; returns callable in many TF-Hub word models
    except Exception as e:
        # fallback to KerasLayer (works for many TF2 hub models)
        module = hub.KerasLayer(handle)

    def embed(words: List[str]) -> np.ndarray:
        # Accepts list of python strings and returns (N, D) float32 numpy array
        try:
            out = module(words)           # often works: module(["hello","world"])
        except Exception:
            # try calling via tf.constant
            out = module(tf.constant(words))
        # ensure numpy float32
        arr = np.asarray(out)
        if arr.dtype != np.float32:
            arr = arr.astype(np.float32)
        return arr

    return embed

#### **GlOVE (via Stanford NLP)**

In [ ]:
def load_glove_from_zip(zip_path: str, dim: int = 100, filename: Optional[str] = None,
                        progress: bool = False) -> Dict[str, np.ndarray]:
    """
    Load glove.6B.{dim}d.txt from a zip (e.g., glove.6B.zip).
    Returns dict: word -> np.float32 vector.
    This avoids gensim and reads directly from the .txt inside the zip.
    """
    if filename is None:
        filename = f"glove.6B.{dim}d.txt"

    embeddings = {}
    with zipfile.ZipFile(zip_path, "r") as zf:
        members = zf.namelist()
        if filename not in members:
            raise FileNotFoundError(f"{filename} not in {zip_path}; available: {members[:10]}")

        with zf.open(filename, "r") as f:
            for i, raw_line in enumerate(f):
                line = raw_line.decode("utf-8", errors="replace").strip()
                if not line:
                    continue
                parts = line.split()
                word = parts[0]
                vals = np.asarray(parts[1:], dtype=np.float32)
                if vals.shape[0] != dim:
                    # skip / warn
                    continue
                embeddings[word] = vals
                if progress and (i % 200000 == 0):
                    print(f"read {i} lines...")

    return embeddings

def build_and_save_glove_matrix(vocab: List[str], glove_dict: Dict[str, np.ndarray],
                                dim: int, out_path: str):
    """
    Build embedding matrix aligned to vocab (list of tokens).
    Unknown tokens get zero vectors. Saves matrix and vocab->idx mapping.
    """
    vocab_lower = [w.lower() for w in vocab]  # match typical glove tokens
    V = len(vocab_lower)
    mat = np.zeros((V, dim), dtype=np.float32)
    missing = []
    for i, w in enumerate(vocab_lower):
        if w in glove_dict:
            mat[i] = glove_dict[w]
        else:
            # try some normalization fallbacks
            w2 = w.replace('-', ' ')
            if w2 in glove_dict:
                mat[i] = glove_dict[w2]
            else:
                missing.append(w)
                # leave zero vector or random init if you prefer:
                # mat[i] = np.random.normal(scale=0.01, size=(dim,)).astype(np.float32)
    # save
    np.save(out_path + ".npy", mat)
    with open(out_path + ".vocab.json", "w", encoding="utf-8") as f:
        json.dump(vocab_lower, f, ensure_ascii=False, indent=2)
    return mat, missing

#### **BERT (via Hugging Face Transformers)**

In [ ]:
class BertEmbedder:
    """A wrapper for Hugging Face's BERT model to generate embeddings."""
    def __init__(self, model_id: str, device: Optional[str] = None):
        self.tokenizer = AutoTokenizer.from_pretrained(model_id)
        self.model = AutoModel.from_pretrained(model_id)
        self.device = torch.device(device or ("cuda" if torch.cuda.is_available() else "cpu"))
        self.model.to(self.device).eval()
        self.dim = self.model.config.hidden_size
        print(f"BERT model loaded on device: {self.device}")

    @torch.no_grad()
    def encode(self, texts: List[str], batch_size=32, max_length=32) -> np.ndarray:
        """Encodes a list of texts into embeddings using mean pooling."""
        all_embeds = []
        for i in range(0, len(texts), batch_size):
            batch = texts[i:i+batch_size]
            inputs = self.tokenizer(batch, padding=True, truncation=True,
                                    max_length=max_length, return_tensors="pt").to(self.device)

            outputs = self.model(**inputs)

            # Perform mean pooling
            mask = inputs["attention_mask"].unsqueeze(-1).expand(outputs.last_hidden_state.size())
            masked_embeddings = outputs.last_hidden_state * mask
            summed = torch.sum(masked_embeddings, 1)
            counts = torch.clamp(mask.sum(1), min=1e-9)
            mean_pooled = summed / counts

            all_embeds.append(mean_pooled.cpu().numpy())

        return np.vstack(all_embeds).astype(np.float32)

# --- Initialize BERT ---
print("\nSetting up BERT...")
BERT_MODEL_ID = "bert-base-uncased"
bert_embedder = BertEmbedder(BERT_MODEL_ID)
print(f"✅ BERT is ready! Model: {BERT_MODEL_ID}, Embedding dimension: {bert_embedder.dim}")


Setting up BERT...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BERT model loaded on device: cpu
✅ BERT is ready! Model: bert-base-uncased, Embedding dimension: 768


#### **Roberta**

In [ ]:
# This is the same class you provided, renamed for generality
class HuggingFaceEmbedder:
    """A generalized wrapper for Hugging Face models to generate phrase embeddings."""
    def __init__(self, model_id: str, device: Optional[str] = None):
        self.tokenizer = AutoTokenizer.from_pretrained(model_id)
        self.model = AutoModel.from_pretrained(model_id)
        self.device = torch.device(device or ("cuda" if torch.cuda.is_available() else "cpu"))
        self.model.to(self.device).eval()
        self.dim = self.model.config.hidden_size
        print(f"Model '{model_id}' loaded on device: {self.device}")

    @torch.no_grad()
    def encode(self, texts: List[str], batch_size=32, max_length=32) -> np.ndarray:
        """Encodes a list of texts into embeddings using mean pooling."""
        all_embeds = []
        for i in range(0, len(texts), batch_size):
            batch = texts[i:i+batch_size]
            inputs = self.tokenizer(batch, padding=True, truncation=True,
                                    max_length=max_length, return_tensors="pt").to(self.device)

            outputs = self.model(**inputs)

            # Perform mean pooling on the last hidden state
            mask = inputs["attention_mask"].unsqueeze(-1)
            summed = (outputs.last_hidden_state * mask).sum(dim=1)
            counts = torch.clamp(mask.sum(dim=1), min=1e-9)
            mean_pooled = summed / counts

            all_embeds.append(mean_pooled.cpu().numpy())

        return np.vstack(all_embeds).astype(np.float32)

# --- Initialize RoBERTa ---
print("Setting up RoBERTa...")
ROBERTA_MODEL_ID = "roberta-base"
roberta_embedder = HuggingFaceEmbedder(ROBERTA_MODEL_ID)
print(f"✅ RoBERTa is ready! Model: {ROBERTA_MODEL_ID}, Embedding dimension: {roberta_embedder.dim}")

Setting up RoBERTa...


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model 'roberta-base' loaded on device: cpu
✅ RoBERTa is ready! Model: roberta-base, Embedding dimension: 768


#### **LaBSE**

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text  # <-- CRITICAL IMPORT: This registers the custom ops
import numpy as np
from typing import List

class LaBSEmbedder:
    """
    A corrected wrapper for the LaBSE model from TensorFlow Hub that ensures
    the necessary pre-processor and its dependencies are loaded correctly.
    """
    def __init__(self, handle: str):
        # Define the handles for the pre-processor and the encoder
        preprocessor_handle = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"

        # Load the pre-processor and the main LaBSE encoder model
        self.preprocessor = hub.KerasLayer(preprocessor_handle)
        self.encoder = hub.KerasLayer(handle, trainable=False)

        # Probe the model to get its output dimension
        test_text = tf.constant(["probing the model"])
        processed_test_text = self.preprocessor(test_text)
        embedding_result = self.encoder(processed_test_text)
        self.dim = int(embedding_result["pooled_output"].shape[-1])

    def encode(self, texts: List[str]) -> np.ndarray:
        """
        Encodes a list of texts by first running them through the pre-processor
        and then the LaBSE encoder.
        """
        text_tensor = tf.constant(texts)
        processed_text = self.preprocessor(text_tensor)
        embedding_result = self.encoder(processed_text)
        return embedding_result["pooled_output"].numpy().astype(np.float32)

# --- Initialize LaBSE with the corrected class ---
print("Setting up LaBSE...")
LABSE_HANDLE = "https://tfhub.dev/google/LaBSE/2"
labse_embedder = LaBSEmbedder(LABSE_HANDLE)
print(f"✅ LaBSE is ready! Embedding dimension: {labse_embedder.dim}")

# --- Example Usage ---
sample_embedding = labse_embedder.encode(["hello world"])
print(f"Successfully created a sample embedding with shape: {sample_embedding.shape}")

Setting up LaBSE...
✅ LaBSE is ready! Embedding dimension: 768
Successfully created a sample embedding with shape: (1, 768)


## Defining Relations Pairs

In [ ]:
# Build relation pairs as provided
def build_relation_pairs() -> Dict[str, List[Tuple[str, str]]]:
    rels: Dict[str, List[Tuple[str, str]]] = {}
    # 1) CITY_IN_STATE (IN, US, CA)
    city_state_pairs = [
        # India (25)
        ("Mumbai","Maharashtra"), ("Pune","Maharashtra"), ("Nagpur","Maharashtra"),
        ("Nashik","Maharashtra"), ("Aurangabad","Maharashtra"),
        ("Bengaluru","Karnataka"), ("Mysuru","Karnataka"), ("Mangaluru","Karnataka"),
        ("Hubballi","Karnataka"), ("Belagavi","Karnataka"),
        ("Chennai","Tamil Nadu"), ("Coimbatore","Tamil Nadu"), ("Madurai","Tamil Nadu"),
        ("Tiruchirappalli","Tamil Nadu"), ("Salem","Tamil Nadu"),
        ("Kolkata","West Bengal"), ("Howrah","West Bengal"), ("Asansol","West Bengal"),
        ("Siliguri","West Bengal"), ("Durgapur","West Bengal"),
        ("Hyderabad","Telangana"), ("Warangal","Telangana"), ("Karimnagar","Telangana"),
        ("Nizamabad","Telangana"), ("Khammam","Telangana"),
        # USA (20)
        ("New York","New York"), ("Buffalo","New York"), ("Albany","New York"),
        ("Los Angeles","California"), ("San Francisco","California"), ("San Diego","California"),
        ("Sacramento","California"), ("Fresno","California"),
        ("Seattle","Washington"), ("Spokane","Washington"),
        ("Chicago","Illinois"), ("Springfield","Illinois"),
        ("Houston","Texas"), ("Dallas","Texas"), ("Austin","Texas"),
        ("Boston","Massachusetts"), ("Worcester","Massachusetts"),
        ("Miami","Florida"), ("Orlando","Florida"), ("Tampa","Florida"),
        # Canada (15)
        ("Toronto","Ontario"), ("Ottawa","Ontario"), ("Mississauga","Ontario"),
        ("Vancouver","British Columbia"), ("Victoria","British Columbia"), ("Surrey","British Columbia"),
        ("Montreal","Quebec"), ("Quebec City","Quebec"), ("Laval","Quebec"),
        ("Calgary","Alberta"), ("Edmonton","Alberta"),
        ("Winnipeg","Manitoba"), ("Saskatoon","Saskatchewan"),
        ("Halifax","Nova Scotia"), ("St. John’s","Newfoundland and Labrador"),
    ]
    assert len(city_state_pairs) == 60
    rels["CITY_IN_STATE"] = city_state_pairs
    # 2) STATE_IN_COUNTRY (IN, US, CA, AU, DE)
    state_country_pairs = [
        # India (17)
        ("Maharashtra","India"), ("Karnataka","India"), ("Tamil Nadu","India"),
        ("West Bengal","India"), ("Telangana","India"), ("Gujarat","India"),
        ("Rajasthan","India"), ("Uttar Pradesh","India"), ("Bihar","India"),
        ("Madhya Pradesh","India"), ("Punjab","India"), ("Haryana","India"),
        ("Odisha","India"), ("Jharkhand","India"), ("Kerala","India"),
        ("Andhra Pradesh","India"), ("Assam","India"),
        # USA (15)
        ("California","USA"), ("Texas","USA"), ("New York","USA"), ("Florida","USA"), ("Illinois","USA"),
        ("Pennsylvania","USA"), ("Ohio","USA"), ("Georgia","USA"), ("North Carolina","USA"), ("Michigan","USA"),
        ("Washington","USA"), ("Massachusetts","USA"), ("Arizona","USA"), ("New Jersey","USA"), ("Virginia","USA"),
        # Canada (10)
        ("Ontario","Canada"), ("Quebec","Canada"), ("British Columbia","Canada"), ("Alberta","Canada"), ("Manitoba","Canada"),
        ("Saskatchewan","Canada"), ("Nova Scotia","Canada"), ("New Brunswick","Canada"),
        ("Newfoundland and Labrador","Canada"), ("Prince Edward Island","Canada"),
        # Australia (8)
        ("New South Wales","Australia"), ("Victoria","Australia"), ("Queensland","Australia"),
        ("Western Australia","Australia"), ("South Australia","Australia"),
        ("Tasmania","Australia"), ("Australian Capital Territory","Australia"),
        ("Northern Territory","Australia"),
        # Germany (10)
        ("Bavaria","Germany"), ("Baden-Württemberg","Germany"), ("North Rhine-Westphalia","Germany"),
        ("Hesse","Germany"), ("Lower Saxony","Germany"), ("Saxony","Germany"),
        ("Berlin","Germany"), ("Hamburg","Germany"), ("Bremen","Germany"), ("Brandenburg","Germany"),
    ]
    assert len(state_country_pairs) == 60
    rels["STATE_IN_COUNTRY"] = state_country_pairs
    # 3) CITY_IN_COUNTRY (for hierarchical test)
    city_country_pairs = [
        ("Mumbai","India"), ("Pune","India"), ("Bengaluru","India"), ("Chennai","India"), ("Kolkata","India"),
        ("Hyderabad","India"), ("Ahmedabad","India"), ("Jaipur","India"), ("Lucknow","India"), ("Kanpur","India"),
        ("Delhi","India"), ("Indore","India"), ("Bhopal","India"), ("Patna","India"), ("Varanasi","India"),
        ("Paris","France"), ("Lyon","France"), ("Marseille","France"), ("Nice","France"), ("Toulouse","France"),
        ("Berlin","Germany"), ("Munich","Germany"), ("Hamburg","Germany"), ("Cologne","Germany"), ("Frankfurt","Germany"),
        ("New York","USA"), ("Los Angeles","USA"), ("San Francisco","USA"), ("Chicago","USA"), ("Seattle","USA"),
        ("Houston","USA"), ("Dallas","USA"), ("Boston","USA"), ("Miami","USA"), ("Atlanta","USA"),
        ("Toronto","Canada"), ("Vancouver","Canada"), ("Montreal","Canada"), ("Calgary","Canada"), ("Ottawa","Canada"),
        ("Tokyo","Japan"), ("Osaka","Japan"), ("Kyoto","Japan"), ("Sapporo","Japan"), ("Yokohama","Japan"),
        ("Beijing","China"), ("Shanghai","China"), ("Shenzhen","China"), ("Guangzhou","China"), ("Chengdu","China"),
        ("Sydney","Australia"), ("Melbourne","Australia"), ("Brisbane","Australia"), ("Perth","Australia"), ("Adelaide","Australia"),
        ("Cairo","Egypt"), ("Istanbul","Turkey"), ("Athens","Greece"), ("Lisbon","Portugal"), ("Madrid","Spain"),
    ]
    assert len(city_country_pairs) == 60
    rels["CITY_IN_COUNTRY"] = city_country_pairs
    # 4) COUNTRY_CAPITAL (for similar)
    country_capital_pairs = [
        ("India","New Delhi"), ("USA","Washington, D.C."), ("Canada","Ottawa"), ("United Kingdom","London"), ("France","Paris"),
        ("Germany","Berlin"), ("Italy","Rome"), ("Spain","Madrid"), ("Portugal","Lisbon"), ("Netherlands","Amsterdam"),
        ("Belgium","Brussels"), ("Switzerland","Bern"), ("Austria","Vienna"), ("Poland","Warsaw"), ("Czech Republic","Prague"),
        ("Hungary","Budapest"), ("Greece","Athens"), ("Turkey","Ankara"), ("Russia","Moscow"), ("Ukraine","Kyiv"),
        ("Norway","Oslo"), ("Sweden","Stockholm"), ("Denmark","Copenhagen"), ("Finland","Helsinki"), ("Iceland","Reykjavik"),
        ("Ireland","Dublin"), ("Australia","Canberra"), ("New Zealand","Wellington"), ("Japan","Tokyo"), ("China","Beijing"),
        ("South Korea","Seoul"), ("North Korea","Pyongyang"), ("Thailand","Bangkok"), ("Vietnam","Hanoi"), ("Malaysia","Kuala Lumpur"),
        ("Singapore","Singapore"), ("Indonesia","Jakarta"), ("Philippines","Manila"), ("Bangladesh","Dhaka"), ("Pakistan","Islamabad"),
        ("Sri Lanka","Sri Jayawardenepura Kotte"), ("Nepal","Kathmandu"), ("Bhutan","Thimphu"), ("Myanmar","Naypyidaw"), ("Saudi Arabia","Riyadh"),
        ("United Arab Emirates","Abu Dhabi"), ("Qatar","Doha"), ("Kuwait","Kuwait City"), ("Oman","Muscat"), ("Iran","Tehran"),
        ("Iraq","Baghdad"), ("Israel","Jerusalem"), ("Egypt","Cairo"), ("South Africa","Pretoria"), ("Kenya","Nairobi"),
        ("Nigeria","Abuja"), ("Ethiopia","Addis Ababa"), ("Morocco","Rabat"), ("Algeria","Algiers"), ("Brazil","Brasília"),
    ]
    assert len(country_capital_pairs) == 60
    rels["COUNTRY_CAPITAL"] = country_capital_pairs
    # 5) COUNTRY_LARGEST_CITY (for similar)
    country_largest_city_pairs = [
        ("India","Mumbai"), ("USA","New York City"), ("Canada","Toronto"), ("United Kingdom","London"), ("France","Paris"),
        ("Germany","Berlin"), ("Italy","Rome"), ("Spain","Madrid"), ("Portugal","Lisbon"), ("Netherlands","Amsterdam"),
        ("Belgium","Antwerp"), ("Switzerland","Zürich"), ("Austria","Vienna"), ("Poland","Warsaw"), ("Czech Republic","Prague"),
        ("Hungary","Budapest"), ("Greece","Athens"), ("Turkey","Istanbul"), ("Russia","Moscow"), ("Ukraine","Kyiv"),
        ("Norway","Oslo"), ("Sweden","Stockholm"), ("Denmark","Copenhagen"), ("Finland","Helsinki"), ("Iceland","Reykjavik"),
        ("Ireland","Dublin"), ("Australia","Sydney"), ("New Zealand","Auckland"), ("Japan","Tokyo"), ("China","Shanghai"),
        ("South Korea","Seoul"), ("North Korea","Pyongyang"), ("Thailand","Bangkok"), ("Vietnam","Ho Chi Minh City"), ("Malaysia","Kuala Lumpur"),
        ("Singapore","Singapore"), ("Indonesia","Jakarta"), ("Philippines","Quezon City"), ("Bangladesh","Dhaka"), ("Pakistan","Karachi"),
        ("Sri Lanka","Colombo"), ("Nepal","Kathmandu"), ("Bhutan","Thimphu"), ("Myanmar","Yangon"), ("Saudi Arabia","Riyadh"),
        ("United Arab Emirates","Dubai"), ("Qatar","Doha"), ("Kuwait","Kuwait City"), ("Oman","Muscat"), ("Iran","Tehran"),
        ("Iraq","Baghdad"), ("Israel","Jerusalem"), ("Egypt","Cairo"), ("South Africa","Johannesburg"), ("Kenya","Nairobi"),
        ("Nigeria","Lagos"), ("Ethiopia","Addis Ababa"), ("Morocco","Casablanca"), ("Algeria","Algiers"), ("Brazil","São Paulo"),
    ]
    assert len(country_largest_city_pairs) == 60
    rels["COUNTRY_LARGEST_CITY"] = country_largest_city_pairs
    # 6) CITY_ON_RIVER (broad, mainstream)
    city_on_river_pairs = [
        ("Cairo","Nile"), ("Khartoum","Nile"), ("Aswan","Nile"), ("Luxor","Nile"), ("Alexandria","Nile Delta"),
        ("London","Thames"), ("Oxford","Thames"), ("Reading","Thames"), ("Windsor","Thames"), ("Kingston upon Thames","Thames"),
        ("Paris","Seine"), ("Rouen","Seine"), ("Le Havre","Seine"), ("Melun","Seine"), ("Troyes","Seine"),
        ("Budapest","Danube"), ("Vienna","Danube"), ("Belgrade","Danube"), ("Bratislava","Danube"), ("Linz","Danube"),
        ("Prague","Vltava"), ("Hamburg","Elbe"), ("Dresden","Elbe"), ("Magdeburg","Elbe"), ("Usti nad Labem","Elbe"),
        ("Cologne","Rhine"), ("Basel","Rhine"), ("Bonn","Rhine"), ("Mainz","Rhine"), ("Düsseldorf","Rhine"),
        ("Amsterdam","Amstel"), ("Rotterdam","Nieuwe Maas"), ("Kolkata","Hooghly"), ("Varanasi","Ganga"), ("Kanpur","Ganga"),
        ("Patna","Ganga"), ("Allahabad","Yamuna"), ("Agra","Yamuna"), ("Delhi","Yamuna"), ("Mathura","Yamuna"),
        ("Rome","Tiber"), ("Turin","Po"), ("Milan","Po"), ("Pisa","Arno"), ("Florence","Arno"),
        ("Seville","Guadalquivir"), ("Porto","Douro"), ("Lisbon","Tagus"), ("Toledo","Tagus"), ("Madrid","Manzanares"),
        ("Warsaw","Vistula"), ("Krakow","Vistula"), ("Moscow","Moskva"), ("Saint Petersburg","Neva"), ("Shanghai","Huangpu"),
        ("Nanjing","Yangtze"), ("Wuhan","Yangtze"), ("Chongqing","Yangtze"), ("New Orleans","Mississippi"), ("St. Louis","Mississippi"),
    ]
    assert len(city_on_river_pairs) == 60
    rels["CITY_ON_RIVER"] = city_on_river_pairs
    # 7) RIVER_FLOWS_INTO (hydrology graph)
    river_flows_into_pairs = [
        ("Nile","Mediterranean Sea"), ("Blue Nile","Nile"), ("White Nile","Nile"),
        ("Thames","North Sea"), ("Seine","English Channel"),
        ("Danube","Black Sea"), ("Vltava","Elbe"), ("Elbe","North Sea"), ("Rhine","North Sea"), ("Main","Rhine"),
        ("Amstel","IJ"), ("Nieuwe Maas","Haringvliet"),
        ("Hooghly","Bay of Bengal"), ("Ganga","Bay of Bengal"), ("Yamuna","Ganga"),
        ("Tiber","Tyrrhenian Sea"), ("Po","Adriatic Sea"), ("Arno","Ligurian Sea"),
        ("Guadalquivir","Atlantic Ocean"), ("Douro","Atlantic Ocean"), ("Tagus","Atlantic Ocean"),
        ("Manzanares","Jarama"), ("Jarama","Tagus"),
        ("Vistula","Baltic Sea"), ("Oder","Baltic Sea"),
        ("Moskva","Oka"), ("Oka","Volga"), ("Volga","Caspian Sea"), ("Neva","Gulf of Finland"),
        ("Huangpu","Yangtze"), ("Yangtze","East China Sea"), ("Qiantang","East China Sea"), ("Grand Canal","Yangtze"),
        ("Mississippi","Gulf of Mexico"), ("Illinois","Mississippi"), ("Missouri","Mississippi"),
        ("Ohio","Mississippi"), ("Allegheny","Ohio"), ("Monongahela","Ohio"),
        ("Cuyahoga","Lake Erie"), ("Detroit River","Lake Erie"), ("St. Clair River","Lake St. Clair"),
        ("Saint Lawrence","Atlantic Ocean"), ("Ottawa River","Saint Lawrence"), ("Red River","Lake Winnipeg"),
        ("Mersey","Irish Sea"), ("Clyde","Firth of Clyde"),
        ("Tyne","North Sea"), ("Wear","North Sea"), ("Tees","North Sea"),
        ("Ebro","Mediterranean Sea"), ("Severn","Bristol Channel"),
        ("Don","Sea of Azov"), ("Dnieper","Black Sea"), ("Dniester","Black Sea"),
        ("Tigris","Shatt al-Arab"), ("Euphrates","Shatt al-Arab"),
        ("Jordan","Dead Sea"),
        ("Indus","Arabian Sea"), ("Brahmaputra","Bay of Bengal"),
    ]
    assert len(river_flows_into_pairs) == 60
    rels["RIVER_FLOWS_INTO"] = river_flows_into_pairs
    # 8) LANGUAGE_FAMILY (for dissimilar baseline)
    language_family_pairs = [
        ("Hindi","Indo-Aryan"), ("Bengali","Indo-Aryan"), ("Marathi","Indo-Aryan"), ("Gujarati","Indo-Aryan"), ("Punjabi","Indo-Aryan"),
        ("Urdu","Indo-Aryan"), ("Sanskrit","Indo-Aryan"), ("Sinhala","Indo-Aryan"), ("Nepali","Indo-Aryan"), ("Assamese","Indo-Aryan"),
        ("English","Germanic"), ("German","Germanic"), ("Dutch","Germanic"), ("Swedish","Germanic"), ("Norwegian","Germanic"),
        ("Danish","Germanic"), ("Icelandic","Germanic"), ("Afrikaans","Germanic"), ("Frisian","Germanic"), ("Yiddish","Germanic"),
        ("French","Romance"), ("Spanish","Romance"), ("Portuguese","Romance"), ("Italian","Romance"), ("Romanian","Romance"),
        ("Catalan","Romance"), ("Galician","Romance"), ("Occitan","Romance"), ("Sardinian","Romance"), ("Corsican","Romance"),
        ("Russian","Slavic"), ("Polish","Slavic"), ("Czech","Slavic"), ("Slovak","Slavic"), ("Ukrainian","Slavic"),
        ("Serbian","Slavic"), ("Croatian","Slavic"), ("Bulgarian","Slavic"), ("Slovene","Slavic"), ("Belarusian","Slavic"),
        ("Greek","Hellenic"),
        ("Turkish","Turkic"), ("Azerbaijani","Turkic"), ("Kazakh","Turkic"), ("Uzbek","Turkic"), ("Turkmen","Turkic"),
        ("Arabic","Semitic"), ("Hebrew","Semitic"), ("Amharic","Semitic"), ("Tigrinya","Semitic"), ("Aramaic","Semitic"),
        ("Persian","Iranian"), ("Pashto","Iranian"), ("Kurdish","Iranian"), ("Farsi","Iranian"), ("Tajik","Iranian"),
        ("Tamil","Dravidian"), ("Telugu","Dravidian"), ("Kannada","Dravidian"), ("Malayalam","Dravidian"), ("Basque","Language isolate"),
    ]
    print(len(language_family_pairs) == 60)
    rels["LANGUAGE_FAMILY"] = language_family_pairs
    return rels

In [ ]:
# --------------------
# Relation groups (between-relations)
# --------------------
RELATION_TYPES = {
    # Hierarchical: A ⇒ B (city→state + state→country ⇒ city→country)
    "hierarchical": [("CITY_IN_STATE", "CITY_IN_COUNTRY")],
    # Similar (same domain & codomain: country→city, different semantics)
    "similar": [("COUNTRY_CAPITAL", "COUNTRY_LARGEST_CITY")],
    # Compositional (A∘B): CITY_IN_STATE ∘ STATE_I  N_COUNTRY
    "compositional": [("CITY_IN_STATE", "STATE_IN_COUNTRY")],
    # Dissimilar (orthogonal domains)
    "dissimilar": [("LANGUAGE_FAMILY", "CITY_ON_RIVER")],  # strong cross-domain contrast
}

## Calculating Eigenspaces for Each Relation Per Embedding

In [ ]:
import os
import math
from collections import defaultdict

# --- Create a new directory for this specific analysis ---
GEO_REPORT_DIR = "./geospatial_pca_reports"
os.makedirs(GEO_REPORT_DIR, exist_ok=True)
print(f"Reports for the new relations will be saved in: {GEO_REPORT_DIR}")

# --- Part 1: Use your function to build the new relation sets ---
NEW_RELATION_SETS = build_relation_pairs()
print(f"Successfully built {len(NEW_RELATION_SETS)} new relation sets: {list(NEW_RELATION_SETS.keys())}")

# --- Part 2: Re-use the analysis functions and pipeline from your script ---
# NOTE: The following helper functions are copied from your provided code.

def l2_normalize_rows(X: np.ndarray, eps: float = 1e-12) -> np.ndarray:
    norms = np.linalg.norm(X, axis=1, keepdims=True)
    return X / np.clip(norms, eps, None)

def safe_center(X: np.ndarray) -> np.ndarray:
    return X - X.mean(axis=0, keepdims=True)

def explained_variance_from_singular_values(s: np.ndarray, n_samples: int) -> np.ndarray:
    lam = (s ** 2) / max(n_samples - 1, 1)
    total = lam.sum() if lam.sum() > 0 else 1.0
    evr = lam / total
    return lam, evr

W2V_READY = 'w2v_embed' in globals() and w2v_embed is not None
GLOVE_READY = 'GLOVE_DICT' in globals() and len(GLOVE_DICT) > 0
BERT_READY = 'bert_embedder' in globals()
ROBERTA_READY = 'roberta_embedder' in globals()
LABSE_READY = 'labse_embedder' in globals()
GLOVE_DIM = 100

EMBEDDING_KINDS = []
if W2V_READY: EMBEDDING_KINDS.append("w2v")
if GLOVE_READY: EMBEDDING_KINDS.append("glove")
if BERT_READY: EMBEDDING_KINDS.append("bert")
if ROBERTA_READY: EMBEDDING_KINDS.append("roberta")
if LABSE_READY: EMBEDDING_KINDS.append("labse")

def embed_words(words: List[str], embedding_kind: str) -> np.ndarray:
    if embedding_kind == "w2v":
        if not W2V_READY: raise RuntimeError("Word2Vec embedder not initialized.")
        return np.asarray(w2v_embed(words), dtype=np.float32)
    if embedding_kind == "glove":
        if not GLOVE_READY: raise RuntimeError("GloVe not loaded.")
        out = np.zeros((len(words), GLOVE_DIM), dtype=np.float32)
        for i, w in enumerate(words):
            vec = GLOVE_DICT.get(w.lower())
            if vec is None: vec = GLOVE_DICT.get(w.lower().replace('-', ' '))
            if vec is not None: out[i] = vec
        return out
    if embedding_kind == "bert":
        if not BERT_READY: raise RuntimeError("BERT embedder not initialized.")
        return bert_embedder.encode(words)
    if embedding_kind == "roberta":
        if not ROBERTA_READY: raise RuntimeError("RoBERTa embedder not initialized.")
        return roberta_embedder.encode(words)
    if embedding_kind == "labse":
        if not LABSE_READY: raise RuntimeError("LaBSE embedder not initialized.")
        return labse_embedder.encode(words)
    raise ValueError(f"Unknown embedding_kind: {embedding_kind}")

K_MAX = 64
VAR_TARGET = 0.95
NORMALIZE_ROWS = True

def relation_pca_for_pairs(pairs: List[Tuple[str, str]],
                           embedding_kind: str,
                           normalize_rows: bool = NORMALIZE_ROWS,
                           k_max: int = K_MAX,
                           var_target: float = VAR_TARGET,
                           drop_zero_rows: bool = True) -> Dict:
    if len(pairs) == 0: raise ValueError("Empty pair list")
    U = [u for (u, v) in pairs]
    V = [v for (u, v) in pairs]
    EU = embed_words(U, embedding_kind)
    EV = embed_words(V, embedding_kind)
    if normalize_rows:
        EU = l2_normalize_rows(EU)
        EV = l2_normalize_rows(EV)
    R = EV - EU
    if drop_zero_rows:
        mask = (np.linalg.norm(EU, axis=1) > 1e-9) & (np.linalg.norm(EV, axis=1) > 1e-9)
        if mask.sum() < len(pairs):
            print(f"[{embedding_kind}] Dropping {(~mask).sum()} pairs due to OOV/zero vectors.")
        R = R[mask]
    n, d = R.shape
    if n < 2: raise ValueError(f"Not enough valid pairs after filtering for {embedding_kind}: n={n}")
    Rc = safe_center(R)
    _, s, Vt = np.linalg.svd(Rc, full_matrices=False)
    lam, evr = explained_variance_from_singular_values(s, n)
    cum_evr = np.cumsum(evr)
    k_eff = min(k_max, len(s))
    k_star = int(np.searchsorted(cum_evr[:k_eff], var_target) + 1)
    if k_star > k_eff: k_star = k_eff
    spectrum = [{"component": i + 1, "singular_value": float(s[i]), "eigenvalue": float(lam[i]),
                 "explained_variance_ratio": float(evr[i]), "cumulative_evr": float(cum_evr[i])}
                for i in range(len(s))]
    return {
        "n_pairs_used": int(n), "embedding_kind": embedding_kind, "embedding_dim": int(d),
        "k_max": int(k_max), "var_target": float(var_target), "k_star": int(k_star),
        "evr_at_k": float(cum_evr[k_star - 1]), "orthogonal_leakage_at_k": float(1.0 - cum_evr[k_star - 1]),
        "spectrum": spectrum, "basis_top_k": Vt[:k_star].copy(),
    }

# --- Part 3: Run the main analysis loop and store eigenspaces ---
print(f"\n[INFO] Running Relation-PCA for new relations on embeddings: {EMBEDDING_KINDS}")

# ✨ NEW: Initialize the dictionary to store the eigenspaces
EIGENSPACES = {}

OVERVIEW_ROWS = []
KSTAR = {}
EVR_AT_K = {}

for rel_name, pairs in NEW_RELATION_SETS.items():
    for emb_kind in EMBEDDING_KINDS:
        try:
            res = relation_pca_for_pairs(
                pairs=pairs,
                embedding_kind=emb_kind,
                normalize_rows=NORMALIZE_ROWS,
                k_max=K_MAX,
                var_target=VAR_TARGET,
                drop_zero_rows=True
            )

            # ✨ NEW: Store the calculated basis (eigenspace) in the dictionary
            if emb_kind not in EIGENSPACES:
                EIGENSPACES[emb_kind] = {}
            EIGENSPACES[emb_kind][rel_name] = res["basis_top_k"]

            # --- Existing logic for reporting and saving ---
            KSTAR[(emb_kind, rel_name)] = res["k_star"]
            EVR_AT_K[(emb_kind, rel_name)] = res["evr_at_k"]

            spec_csv = os.path.join(GEO_REPORT_DIR, f"spectrum_{emb_kind}_{rel_name}.csv")
            pd.DataFrame(res['spectrum']).to_csv(spec_csv, index=False)

            compact = {k: v for k, v in res.items() if k not in ("basis_top_k",)}
            json_path = os.path.join(GEO_REPORT_DIR, f"summary_{emb_kind}_{rel_name}.json")
            with open(json_path, "w", encoding="utf-8") as jf:
                json.dump(compact, jf, ensure_ascii=False, indent=2)

            basis_path = os.path.join(GEO_REPORT_DIR, f"basis_{emb_kind}_{rel_name}_topk.npy")
            np.save(basis_path, res["basis_top_k"])

            OVERVIEW_ROWS.append({
                "relation": rel_name, "embedding": emb_kind,
                "n_pairs_used": res["n_pairs_used"], "embedding_dim": res["embedding_dim"],
                "k_star": res["k_star"], "evr_at_k": res["evr_at_k"],
            })
            print(f"[OK] {emb_kind:8s} | {rel_name:25s} | k*={res['k_star']:2d} | EVR={res['evr_at_k']:.3f} | Stored basis shape: {res['basis_top_k'].shape}")

        except Exception as e:
            print(f"[FAIL] {emb_kind} | {rel_name}: {e}")

# --- Part 4: Generate and save summary tables (unchanged) ---
if OVERVIEW_ROWS:
    pretty_cells = {emb: {rel: f"k={KSTAR.get((emb, rel), '?')}, evr={EVR_AT_K.get((emb, rel), 0):.2f}" if (emb, rel) in KSTAR else "—" for rel in NEW_RELATION_SETS.keys()} for emb in EMBEDDING_KINDS}
    pretty_df = pd.DataFrame.from_dict(pretty_cells, orient="index", columns=NEW_RELATION_SETS.keys())
    pretty_df.index.name = "embedding"
    pretty_csv = os.path.join(GEO_REPORT_DIR, "table_pretty_metrics.csv")
    pretty_df.to_csv(pretty_csv)

    print("\n=== Relation-PCA Key Metrics for New Relations ===")
    print(pretty_df.to_string(max_cols=120, justify="center"))
    print(f"\n[INFO] Saved summary tables and reports to {GEO_REPORT_DIR}")
else:
    print("[WARN] No results were generated to tabulate.")

# --- Part 5: Verify that eigenspaces are stored ---
print("\n--- Eigenspace Storage Verification ---")
if EIGENSPACES:
    first_emb_key = next(iter(EIGENSPACES.keys()))
    print(f"✅ Successfully stored eigenspaces for {len(EIGENSPACES)} embedding models: {list(EIGENSPACES.keys())}")

    first_rel_key = next(iter(EIGENSPACES[first_emb_key].keys()))
    print(f"For '{first_emb_key}', stored subspaces for {len(EIGENSPACES[first_emb_key])} relations.")

    sample_space = EIGENSPACES[first_emb_key][first_rel_key]
    print(f"Shape of a sample eigenspace ('{first_emb_key}' | '{first_rel_key}'): {sample_space.shape} -> (k*, embedding_dim)")
    print("\nThe variable 'EIGENSPACES' is now ready for your subspace similarity analysis.")
else:
    print("❌ No eigenspaces were stored. Check for errors in the analysis loop.")

Reports for the new relations will be saved in: ./geospatial_pca_reports
False
Successfully built 8 new relation sets: ['CITY_IN_STATE', 'STATE_IN_COUNTRY', 'CITY_IN_COUNTRY', 'COUNTRY_CAPITAL', 'COUNTRY_LARGEST_CITY', 'CITY_ON_RIVER', 'RIVER_FLOWS_INTO', 'LANGUAGE_FAMILY']

[INFO] Running Relation-PCA for new relations on embeddings: ['bert', 'roberta', 'labse']
[OK] bert     | CITY_IN_STATE             | k*=40 | EVR=0.954 | Stored basis shape: (40, 768)
[OK] roberta  | CITY_IN_STATE             | k*=40 | EVR=0.954 | Stored basis shape: (40, 768)
[OK] labse    | CITY_IN_STATE             | k*=41 | EVR=0.951 | Stored basis shape: (41, 768)
[OK] bert     | STATE_IN_COUNTRY          | k*=39 | EVR=0.951 | Stored basis shape: (39, 768)
[OK] roberta  | STATE_IN_COUNTRY          | k*=39 | EVR=0.953 | Stored basis shape: (39, 768)
[OK] labse    | STATE_IN_COUNTRY          | k*=41 | EVR=0.953 | Stored basis shape: (41, 768)
[OK] bert     | CITY_IN_COUNTRY           | k*=41 | EVR=0.953 | Stored

In [ ]:
import numpy as np
import pandas as pd
from scipy.linalg import svd

# --- 1. Define Subspace Similarity Metrics ---

def calculate_grassmann_distance(A: np.ndarray, B: np.ndarray) -> tuple[float, int]:
    """
    Calculates the Grassmann distance between two subspaces represented by orthonormal bases A and B.

    The distance is the L2-norm of the vector of principal angles between the subspaces.

    Args:
        A (np.ndarray): Orthonormal basis for the first subspace, shape (k1, d).
        B (np.ndarray): Orthonormal basis for the second subspace, shape (k2, d).

    Returns:
        A tuple containing:
        - The Grassmann distance (float).
        - The dimension k used for comparison (int).
    """
    # The number of principal angles is determined by the smaller rank.
    # The bases from PCA are already ordered by importance, so we truncate.
    k = min(A.shape[0], B.shape[0])
    A_trunc = A[:k, :]
    B_trunc = B[:k, :]

    # The singular values of the matrix product are the cosines of the principal angles.
    # We compute svd(A_trunc @ B_trunc.T), which is a (k, k) matrix.
    s = svd(A_trunc @ B_trunc.T, compute_uv=False)

    # Clip values to the [-1, 1] range for arccos due to potential numerical inaccuracies.
    s = np.clip(s, -1.0, 1.0)

    # Principal angles are the arccos of the singular values.
    thetas = np.arccos(s)

    # The Grassmann distance is the 2-norm of the vector of principal angles.
    distance = np.linalg.norm(thetas)

    return distance, k

def calculate_projection_frobenius_distance(A: np.ndarray, B: np.ndarray) -> tuple[float, int]:
    """
    Calculates the Frobenius norm of the difference between the projection matrices of two subspaces.

    Args:
        A (np.ndarray): Orthonormal basis for the first subspace, shape (k1, d).
        B (np.ndarray): Orthonormal basis for the second subspace, shape (k2, d).

    Returns:
        A tuple containing:
        - The Frobenius distance (float).
        - The dimension k used for comparison (int).
    """
    # Truncate to the same rank for a fair comparison.
    k = min(A.shape[0], B.shape[0])
    A_trunc = A[:k, :]
    B_trunc = B[:k, :]

    # Projection matrix P_A = A.T @ A, which results in a (d, d) matrix.
    P_A = A_trunc.T @ A_trunc
    P_B = B_trunc.T @ B_trunc

    # Calculate the Frobenius norm of the difference.
    distance = np.linalg.norm(P_A - P_B, ord='fro')

    return distance, k

# --- 2. Iterate and Calculate Similarities ---

# Check if the EIGENSPACES variable exists
if 'EIGENSPACES' not in globals() or not EIGENSPACES:
    raise NameError("The 'EIGENSPACES' dictionary is not defined. Please run the PCA analysis script first.")

similarity_results = []

print("--- Calculating Subspace Similarities ---")

# Iterate over each embedding model we have results for
for emb_kind, rel_spaces in EIGENSPACES.items():
    print(f"\nProcessing Embedding: {emb_kind.upper()}")

    # Iterate over the types of relationships to compare (hierarchical, similar, etc.)
    for rel_type, pairs_to_compare in RELATION_TYPES.items():

        # Iterate over each specific pair within that type
        for rel_A_name, rel_B_name in pairs_to_compare:

            # Ensure both eigenspaces were successfully computed and stored
            if rel_A_name in rel_spaces and rel_B_name in rel_spaces:
                basis_A = rel_spaces[rel_A_name]
                basis_B = rel_spaces[rel_B_name]

                # --- Calculate the two similarity metrics ---
                grassmann_dist, compared_k_grassmann = calculate_grassmann_distance(basis_A, basis_B)
                proj_frob_dist, compared_k_frobenius = calculate_projection_frobenius_distance(basis_A, basis_B)

                # Store the results
                similarity_results.append({
                    "embedding": emb_kind,
                    "relation_type": rel_type,
                    "relation_A": rel_A_name,
                    "k_A": basis_A.shape[0],
                    "relation_B": rel_B_name,
                    "k_B": basis_B.shape[0],
                    "compared_k": compared_k_grassmann, # k is the same for both methods
                    "grassmann_distance": grassmann_dist,
                    "projection_frobenius_distance": proj_frob_dist
                })
                print(f"  - Compared '{rel_A_name}' vs '{rel_B_name}' ({rel_type})")
            else:
                print(f"  - Skipping '{rel_A_name}' vs '{rel_B_name}': one or both eigenspaces missing.")

# --- 3. Display Results in a DataFrame ---

if not similarity_results:
    print("\nNo similarity results were generated. Please check for errors.")
else:
    results_df = pd.DataFrame(similarity_results)

    # Set display options for better readability
    pd.set_option('display.float_format', '{:.4f}'.format)
    pd.set_option('display.max_rows', 100)
    pd.set_option('display.width', 120)

    print("\n--- Subspace Similarity Analysis Results ---")
    print(results_df)

    # Save the results to a CSV file for further analysis
    output_path = "./geospatial_pca_reports/subspace_similarity_results.csv"
    results_df.to_csv(output_path, index=False)
    print(f"\n[INFO] Results have been saved to: {output_path}")

--- Calculating Subspace Similarities ---

Processing Embedding: BERT
  - Compared 'CITY_IN_STATE' vs 'CITY_IN_COUNTRY' (hierarchical)
  - Compared 'COUNTRY_CAPITAL' vs 'COUNTRY_LARGEST_CITY' (similar)
  - Compared 'CITY_IN_STATE' vs 'STATE_IN_COUNTRY' (compositional)
  - Compared 'LANGUAGE_FAMILY' vs 'CITY_ON_RIVER' (dissimilar)

Processing Embedding: ROBERTA
  - Compared 'CITY_IN_STATE' vs 'CITY_IN_COUNTRY' (hierarchical)
  - Compared 'COUNTRY_CAPITAL' vs 'COUNTRY_LARGEST_CITY' (similar)
  - Compared 'CITY_IN_STATE' vs 'STATE_IN_COUNTRY' (compositional)
  - Compared 'LANGUAGE_FAMILY' vs 'CITY_ON_RIVER' (dissimilar)

Processing Embedding: LABSE
  - Compared 'CITY_IN_STATE' vs 'CITY_IN_COUNTRY' (hierarchical)
  - Compared 'COUNTRY_CAPITAL' vs 'COUNTRY_LARGEST_CITY' (similar)
  - Compared 'CITY_IN_STATE' vs 'STATE_IN_COUNTRY' (compositional)
  - Compared 'LANGUAGE_FAMILY' vs 'CITY_ON_RIVER' (dissimilar)

--- Subspace Similarity Analysis Results ---
   embedding  relation_type       rela

In [ ]:
import numpy as np
import pandas as pd
from scipy.linalg import svd

# --- 1. Define New Subspace Comparison Metrics ---

def calculate_containment_score(A: np.ndarray, B: np.ndarray) -> float:
    """
    Calculates the containment score C(A -> B).
    Measures how much of subspace A is contained within subspace B.
    """
    # A has shape (k_A, d), B has shape (k_B, d)
    k_A = A.shape[0]
    if k_A == 0:
        return 0.0

    # ||A @ B.T||_F^2 is the sum of squared cosines of principal angles
    # between A and the projection of B onto A.
    sum_sq_cosines = np.sum(np.linalg.svd(A @ B.T, compute_uv=False)**2)

    return sum_sq_cosines / k_A

def calculate_asymmetric_distance(A: np.ndarray, B: np.ndarray) -> float:
    """
    Calculates the distance d(A -> B).
    Measures the norm of the part of A's basis that is orthogonal to subspace B.
    """
    # Projection matrix for B is P_B = B.T @ B
    P_B = B.T @ B

    # The residual matrix contains the parts of A's basis vectors
    # that are orthogonal to B. A is (k_A, d), P_B is (d, d)
    # The basis vectors of A are rows, so we work with A.T (d, k_A)
    residual_matrix = A.T - P_B @ A.T

    return np.linalg.norm(residual_matrix, ord='fro')

def calculate_max_principal_angle(A: np.ndarray, B: np.ndarray, degrees: bool = True) -> float:
    """
    Calculates the largest principal angle (tilt) between subspaces A and B.
    """
    # The smallest singular value corresponds to the largest angle.
    s = svd(A @ B.T, compute_uv=False)

    # Clip for numerical stability with arccos
    min_s = np.clip(np.min(s), 0.0, 1.0)

    max_angle_rad = np.arccos(min_s)

    return np.rad2deg(max_angle_rad) if degrees else max_angle_rad

# --- 2. Iterate and Calculate New Metrics ---

if 'EIGENSPACES' not in globals() or not EIGENSPACES:
    raise NameError("The 'EIGENSPACES' dictionary is not defined. Please run the PCA analysis script first.")

new_metrics_results = []
print("\n--- Calculating Containment and Tilt Metrics ---")

for emb_kind, rel_spaces in EIGENSPACES.items():
    print(f"\nProcessing Embedding: {emb_kind.upper()}")

    for rel_type, pairs_to_compare in RELATION_TYPES.items():
        for rel_A_name, rel_B_name in pairs_to_compare:
            if rel_A_name in rel_spaces and rel_B_name in rel_spaces:
                basis_A = rel_spaces[rel_A_name]
                basis_B = rel_spaces[rel_B_name]

                # --- Calculate all new metrics ---
                # Note: Containment and Asymmetric Distance are directional
                containment_A_in_B = calculate_containment_score(basis_A, basis_B)
                containment_B_in_A = calculate_containment_score(basis_B, basis_A)

                distance_A_outside_B = calculate_asymmetric_distance(basis_A, basis_B)
                distance_B_outside_A = calculate_asymmetric_distance(basis_B, basis_A)

                tilt = calculate_max_principal_angle(basis_A, basis_B)

                new_metrics_results.append({
                    "embedding": emb_kind,
                    "relation_type": rel_type,
                    "rel_A": rel_A_name,
                    "rel_B": rel_B_name,
                    "containment(A->B)": containment_A_in_B,
                    "containment(B->A)": containment_B_in_A,
                    "dist(A_outside_B)": distance_A_outside_B,
                    "dist(B_outside_A)": distance_B_outside_A,
                    "max_angle(tilt)": tilt
                })
                print(f"  - Analyzed '{rel_A_name}' vs '{rel_B_name}' ({rel_type})")

# --- 3. Display Results in a DataFrame ---
if not new_metrics_results:
    print("\nNo new metrics were generated. Please check for errors.")
else:
    new_results_df = pd.DataFrame(new_metrics_results)

    print("\n--- Subspace Containment and Tilt Analysis Results ---")
    print(new_results_df)

    # Save the results to a new CSV file
    output_path = "./geospatial_pca_reports/subspace_containment_tilt_results.csv"
    new_results_df.to_csv(output_path, index=False)
    print(f"\n[INFO] Containment and tilt results saved to: {output_path}")


--- Calculating Containment and Tilt Metrics ---

Processing Embedding: BERT
  - Analyzed 'CITY_IN_STATE' vs 'CITY_IN_COUNTRY' (hierarchical)
  - Analyzed 'COUNTRY_CAPITAL' vs 'COUNTRY_LARGEST_CITY' (similar)
  - Analyzed 'CITY_IN_STATE' vs 'STATE_IN_COUNTRY' (compositional)
  - Analyzed 'LANGUAGE_FAMILY' vs 'CITY_ON_RIVER' (dissimilar)

Processing Embedding: ROBERTA
  - Analyzed 'CITY_IN_STATE' vs 'CITY_IN_COUNTRY' (hierarchical)
  - Analyzed 'COUNTRY_CAPITAL' vs 'COUNTRY_LARGEST_CITY' (similar)
  - Analyzed 'CITY_IN_STATE' vs 'STATE_IN_COUNTRY' (compositional)
  - Analyzed 'LANGUAGE_FAMILY' vs 'CITY_ON_RIVER' (dissimilar)

Processing Embedding: LABSE
  - Analyzed 'CITY_IN_STATE' vs 'CITY_IN_COUNTRY' (hierarchical)
  - Analyzed 'COUNTRY_CAPITAL' vs 'COUNTRY_LARGEST_CITY' (similar)
  - Analyzed 'CITY_IN_STATE' vs 'STATE_IN_COUNTRY' (compositional)
  - Analyzed 'LANGUAGE_FAMILY' vs 'CITY_ON_RIVER' (dissimilar)

--- Subspace Containment and Tilt Analysis Results ---
   embedding  relati

In [ ]:
import numpy as np
import pandas as pd

# --- Helper Functions ---

def get_relation_vectors(relation_name: str, emb_kind: str) -> np.ndarray:
    """
    Embeds all pairs for a given relation and returns the matrix of difference vectors.
    """
    pairs = NEW_RELATION_SETS[relation_name]
    # Drop OOV pairs to ensure consistency, using the same logic as the PCA step
    U = [u for u, v in pairs]
    V = [v for u, v in pairs]
    EU = embed_words(U, emb_kind)
    EV = embed_words(V, emb_kind)

    # Filter out zero-norm vectors which indicate OOV words
    mask = (np.linalg.norm(EU, axis=1) > 1e-9) & (np.linalg.norm(EV, axis=1) > 1e-9)

    if (~mask).sum() > 0:
        print(f"  [NOTE] Dropping {(~mask).sum()} OOV pairs from '{relation_name}' for '{emb_kind}'.")

    return EV[mask] - EU[mask]

def cosine_similarity(v1: np.ndarray, v2: np.ndarray) -> float:
    """Calculates cosine similarity between two vectors."""
    v1_norm = np.linalg.norm(v1)
    v2_norm = np.linalg.norm(v2)
    if v1_norm == 0 or v2_norm == 0:
        return 0.0
    return np.dot(v1, v2) / (v1_norm * v2_norm)

# --- Check for prerequisites ---
if 'NEW_RELATION_SETS' not in globals() or 'EMBEDDING_KINDS' not in globals():
    raise NameError("Required variables are not defined. Please run the previous analysis cells first.")

# ===================================================================
# Experiment 1: Test for Additive Composition of Relation Vectors
# ===================================================================
print("--- Experiment 1: Testing Additive Composition ---")
print("Hypothesis: mean(CITY->STATE) + mean(STATE->COUNTRY) ≈ mean(CITY->COUNTRY)\n")

composition_results = []

for emb_kind in EMBEDDING_KINDS:
    try:
        print(f"Processing Embedding: {emb_kind.upper()}")

        # 1. Get the matrices of difference vectors for the three relations
        vecs_CS = get_relation_vectors("CITY_IN_STATE", emb_kind)
        vecs_SC = get_relation_vectors("STATE_IN_COUNTRY", emb_kind)
        vecs_CC = get_relation_vectors("CITY_IN_COUNTRY", emb_kind)

        # 2. Calculate the mean vector (centroid) for each relation
        r_CS = np.mean(vecs_CS, axis=0)
        r_SC = np.mean(vecs_SC, axis=0)
        r_CC_actual = np.mean(vecs_CC, axis=0)

        # 3. Add the constituent vectors to create the composed vector
        r_composed = r_CS + r_SC

        # 4. Compare the composed vector to the actual target vector
        similarity = cosine_similarity(r_composed, r_CC_actual)

        composition_results.append({
            "embedding": emb_kind,
            "similarity(composed, actual)": similarity
        })
        print(f"  - Cosine Similarity: {similarity:.4f}")

    except Exception as e:
        print(f"  - FAILED for {emb_kind}: {e}")

# Display results for Experiment 1
comp_df = pd.DataFrame(composition_results)
print("\n--- Additive Composition Results Summary ---")
print(comp_df)


# ===================================================================
# Experiment 2: Train and Evaluate a Linear Probe
# ===================================================================
print("\n\n--- Experiment 2: Probing with a Linear Transformation ---")
print("Hypothesis: A linear map W exists such that W(CITY->STATE) ≈ (STATE->COUNTRY)\n")

probe_results = []

for emb_kind in EMBEDDING_KINDS:
    try:
        print(f"Processing Embedding: {emb_kind.upper()}")

        # 1. Get the source (X) and target (Y) matrices of difference vectors
        X = get_relation_vectors("CITY_IN_STATE", emb_kind)
        Y = get_relation_vectors("STATE_IN_COUNTRY", emb_kind)

        # For a valid regression, we need to ensure we can match pairs.
        # Since the vocab doesn't overlap, we treat the sets of vectors as distributions.
        # We find a transformation W that maps the entire source cloud X to the target cloud Y.
        # We need to ensure X and Y have the same number of samples for this simple setup.
        n_samples = min(X.shape[0], Y.shape[0])
        X = X[:n_samples]
        Y = Y[:n_samples]

        # 2. Solve for the transformation matrix W using least squares
        # This finds W that minimizes ||Y - XW||^2
        W, residuals, rank, s = np.linalg.lstsq(X, Y, rcond=None)

        # 3. Apply the transformation to predict the target vectors
        Y_pred = X @ W

        # 4. Evaluate the transformation's performance
        # Mean Squared Error (MSE) of the prediction
        mse = np.mean(np.sum((Y - Y_pred)**2, axis=1))

        # Compare MSE to the original variance of the target vectors
        # This gives an R^2-like score: 1 is a perfect fit, 0 is no better than guessing the mean.
        target_variance = np.mean(np.sum((Y - np.mean(Y, axis=0))**2, axis=1))

        explained_variance_score = 1 - (mse / target_variance)

        probe_results.append({
            "embedding": emb_kind,
            "mse": mse,
            "target_variance": target_variance,
            "explained_variance_score": explained_variance_score
        })
        print(f"  - Explained Variance Score (R²-like): {explained_variance_score:.4f}")

    except Exception as e:
        print(f"  - FAILED for {emb_kind}: {e}")

# Display results for Experiment 2
probe_df = pd.DataFrame(probe_results)
print("\n--- Linear Probe Results Summary ---")
print(probe_df)

--- Experiment 1: Testing Additive Composition ---
Hypothesis: mean(CITY->STATE) + mean(STATE->COUNTRY) ≈ mean(CITY->COUNTRY)

Processing Embedding: BERT
  - Cosine Similarity: 0.8333
Processing Embedding: ROBERTA
  - Cosine Similarity: 0.9491
Processing Embedding: LABSE
  - Cosine Similarity: 0.8248

--- Additive Composition Results Summary ---
  embedding  similarity(composed, actual)
0      bert                        0.8333
1   roberta                        0.9491
2     labse                        0.8248


--- Experiment 2: Probing with a Linear Transformation ---
Hypothesis: A linear map W exists such that W(CITY->STATE) ≈ (STATE->COUNTRY)

Processing Embedding: BERT
  - Explained Variance Score (R²-like): 0.9997
Processing Embedding: ROBERTA
  - Explained Variance Score (R²-like): 0.9996
Processing Embedding: LABSE
  - Explained Variance Score (R²-like): 0.9890

--- Linear Probe Results Summary ---
  embedding    mse  target_variance  explained_variance_score
0      bert 0.0074